## Reserve and Start a Server

Reserver and Start a Server

#### Modules 

- [Reserve Mulitple Resources](../modules-python/reservations/reserve_multiple_resources.ipynb)
- [Get Lease by Name](../modules-python/reservations/get_lease_by_name.ipynb)
- [Get Reservation](../modules-python/reservations/get_reservations_from_lease.ipynb)
- [Create Server](../modules-python/servers/create_server.ipynb)
- [Delete Server](../modules-python/servers/delete_server.ipynb)

TODO: add the rest of the module links when they are ready

#### Import Library

```
import chi
from chi.server_api_examples import *
from chi.reservation_api_examples import *
from chi.networking_api_examples import *
```




## Tutorial: 

#### Configure the Environment

Import the chi example API calls, set the project name and region, and set various names and attributes to use in the tutorial. 

In [1]:
import json
import os
import chi

from chi.server_api_examples import *
from chi.reservation_api_examples import *
from chi.networking_api_examples import *

#Config with your project
chi.set('project_name', 'ExoGENI@Chameleon') # Replace with your project name

#Set the region
chi.set('region_name', 'CHI@TACC')     # Optional, defaults to 'CHI@UC'

#Insert keypair name
key_name = 'pruth-jupyter'  # Change to your keypair
key_path = '/home/pruth/work/pruth-jupyter' #path to your private key

# Tip: Name resources with your username for easier identification
username = os.getenv("USER")
server_name = username+'Server'
network_name = 'sharednet1'
stitched_network_name = network_name+"Stitched"
lease_name_servers = username+'LeaseServers'


#Server Config
image_name='CC-CentOS7'
flavor_name='baremetal'
node_type="compute_haswell"
server_count=1


## Create the Servers

In [2]:
# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
add_node_reservation(reservation_list, count=server_count, node_type=node_type)
add_fip_reservation(reservation_list, count=1)

# Create the lease
chi.blazar().lease.create(name=lease_name_servers, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])

#Get the lease by name
server_lease = get_lease_by_name(lease_name_servers)
    
#Print the lease info
print(json.dumps(server_lease, indent=2))

{
  "status": "PENDING",
  "user_id": "f3a422ce5a4442e3a7a247ddfd98bfc3",
  "name": "pruthLeaseServers",
  "end_date": "2020-07-24T17:21:00.000000",
  "reservations": [
    {
      "status": "pending",
      "before_end": "default",
      "lease_id": "e65e6fb6-2a9b-4ba5-a551-c084f04bad1c",
      "resource_id": "64c1f047-6d5b-45f6-9587-5a7ef0e56898",
      "max": 1,
      "created_at": "2020-07-23 17:21:42",
      "min": 1,
      "updated_at": "2020-07-23 17:21:57",
      "missing_resources": false,
      "hypervisor_properties": "",
      "on_start": "default",
      "resource_properties": "[\"==\", \"$node_type\", \"compute_haswell\"]",
      "id": "bbc02c25-4c03-4c81-b6d8-a14e08d400ab",
      "resource_type": "physical:host",
      "resources_changed": false
    },
    {
      "status": "pending",
      "lease_id": "e65e6fb6-2a9b-4ba5-a551-c084f04bad1c",
      "resource_id": "c71e5bf6-e152-4c0f-9265-60f3c936076d",
      "network_id": "6189521e-06a0-4c43-b163-16cc11ce675b",
      "cre

In [3]:
#Get the lease by name
server_lease = get_lease_by_name(lease_name_servers)

compute_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'physical:host', server_lease['reservations']))[0]['id']
floatingip_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'virtual:floatingip', server_lease['reservations']))[0]['id']

print("compute_reservation_id: " + compute_reservation_id)
print("floatingip_reservation_id: " + floatingip_reservation_id)

compute_reservation_id: bbc02c25-4c03-4c81-b6d8-a14e08d400ab
floatingip_reservation_id: fb239cde-a2ce-4d63-a2ef-95c6d2ff0a13


#### Start the Server

Use the compute_reservation_id to [create the server](../modules-python/servers/create_server.ipynb).

In [4]:
#create the server
server = create_server(server_name, 
                       reservation_id=compute_reservation_id, 
                       key_name=key_name, 
                       network_name=network_name, 
                       image_name=image_name, 
                       flavor_name=flavor_name)


#### Associate the Floating IP   
TODO: need to find floating_ip from the reservation that was just made

In [5]:
floating_ip = associate_floating_ip(server_name)

print('Floating IP: ' + str(floating_ip))

Floating IP: 129.114.109.233


#### Configure Server

In [6]:
script= '#!/bin/bash'   '\n' \
    '{'   '\n' \
    'yum install -y vim'   '\n' \
    'echo Add Config Script Here!'  '\n' \
    '} > /tmp/boot.log 2>&1'   '\n' 

In [9]:
import paramiko 

key = paramiko.RSAKey.from_private_key_file(key_path)
client = paramiko.SSHClient()
client.load_system_host_keys()
client.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())

client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(floating_ip,username='cc',pkey = key)

stdin, stdout, stderr = client.exec_command('echo \"' + script + '\" > script.sh; chmod +x script.sh; sudo ./script.sh')
print ('stdout: ' + str(stdout.read()))
print ('stderr: ' + str(stderr.read()))

client.close()
print ('done!')

stdout: b''
stderr: b''
done!


## Clean Up Resources

### Delete Resources

[Delete the server](../modules-python/servers/delete_server.ipynb) using its name.

In [10]:
delete_server_by_name(server_name)

()

#### Release Leases

In [11]:
delete_lease_by_name(lease_name_servers)

Deleted lease pruthLeaseServers with id e65e6fb6-2a9b-4ba5-a551-c084f04bad1c
